# --- Day 14: Restroom Redoubt ---

To get The Historian safely to the bathroom, you'll need a way to predict where the robots will be in the future. Fortunately, they all seem to be moving on the tile floor in predictable straight lines.

You make a list (your puzzle input) of all of the robots' current positions (p) and velocities (v), one robot per line. For example:

```
p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3
```
Each robot's position is given as p=x,y where x represents the number of tiles the robot is from the left wall and y represents the number of tiles from the top wall (when viewed from above). So, a position of p=0,0 means the robot is all the way in the top-left corner.

Each robot's velocity is given as v=x,y where x and y are given in tiles per second. Positive x means the robot is moving to the right, and positive y means the robot is moving down. So, a velocity of v=1,-2 means that each second, the robot moves 1 tile to the right and 2 tiles up.

The robots outside the actual bathroom are in a space which is 101 tiles wide and 103 tiles tall (when viewed from above). However, in this example, the robots are in a space which is only 11 tiles wide and 7 tiles tall.

The robots are good at navigating over/under each other (due to a combination of springs, extendable legs, and quadcopters), so they can share the same tile and don't interact with each other. Visually, the number of robots on each tile in this example looks like this:

```
1.12.......
...........
...........
......11.11
1.1........
.........1.
.......1...
```
These robots have a unique feature for maximum bathroom security: they can teleport. When a robot would run into an edge of the space they're in, they instead teleport to the other side, effectively wrapping around the edges. Here is what robot p=2,4 v=2,-3 does for the first few seconds:

Initial state:
```
...........
...........
...........
...........
..1........
...........
...........
```

After 1 second:
```
...........
....1......
...........
...........
...........
...........
...........
```

After 2 seconds:
```
...........
...........
...........
...........
...........
......1....
...........
```

After 3 seconds:
```
...........
...........
........1..
...........
...........
...........
...........
```

After 4 seconds:
```
...........
...........
...........
...........
...........
...........
..........1
```
After 5 seconds:
```
...........
...........
...........
.1.........
...........
...........
...........
```
The Historian can't wait much longer, so you don't have to simulate the robots for very long. Where will the robots be after 100 seconds?

In the above example, the number of robots on each tile after 100 seconds has elapsed looks like this:
```
......2..1.
...........
1..........
.11........
.....1.....
...12......
.1....1....
```
To determine the safest area, count the number of robots in each quadrant after 100 seconds. Robots that are exactly in the middle (horizontally or vertically) don't count as being in any quadrant, so the only relevant robots are:
```
..... 2..1.
..... .....
1.... .....
           
..... .....
...12 .....
.1... 1....
```
In this example, the quadrants contain 1, 3, 4, and 1 robot. Multiplying these together gives a total safety factor of 12.

Predict the motion of the robots in your list within a space which is 101 tiles wide and 103 tiles tall. What will the safety factor be after exactly 100 seconds have elapsed?

In [ ]:
from fastcore.utils import L
from aocd import get_data

inp = get_data(day=14, year=2024)
samp = '''p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3'''
samp

Restate the essential parts of the problem.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at this problem, here are the essential parts:

**Setup:**
- Robots move on a grid that's 101 tiles wide and 103 tiles tall
- Each robot has a position (p=x,y) and velocity (v=x,y)
- Robots move in straight lines at constant velocity

**Key mechanics:**
- The grid wraps around (teleportation) - when a robot goes off one edge, it appears on the opposite edge
- Robots can occupy the same tile without interacting

**The task:**
- Simulate the robots' positions after exactly 100 seconds
- Divide the grid into 4 quadrants (ignoring the middle row and middle column)
- Count how many robots are in each quadrant
- Multiply these four counts together to get the "safety factor"

**What you need to figure out:**
- How to calculate each robot's position after 100 seconds with wrapping
- How to determine which quadrant each robot is in (excluding middle lines)
- The final safety factor

Does this match your understanding? What part would you like to tackle first?

Here are my observations:
- First, I don't need to simulate each step - it should be fine to multiply the velocity and direction (using modulo to wrap around edges) of each robot and therefore we get the position after given number of steps.
- Second, the position of each robot can be computed in parallel.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Excellent observations! You're absolutely right on both counts.

**For the first point:** Yes, since the robots move in straight lines at constant velocity, you can directly calculate the final position. If a robot starts at position (x, y) with velocity (vx, vy), after 100 seconds it will be at:
- `(x + 100*vx, y + 100*vy)`

And then you use modulo to handle the wrapping. What modulo operation would you use for the x-coordinate on a grid that's 101 tiles wide?

**For the second point:** Exactly - each robot's movement is independent, so you can process them all separately (or in parallel if you wanted to optimize).

Do you want to start writing the code to parse the input and calculate the final positions?

In [ ]:
xs = samp.splitlines()
xs

In [ ]:
import re
res = re.search(r'p=(\d+),(\d+) v=([^,]+),(.+)', xs[0])
res.group(1), res.group(2), res.group(3), res.group(4)

In [ ]:
res = re.search(r'p=(\d+),(\d+) v=([^,]+),(.+)', xs[0])
lst = L(range(1, 5)).map(res.group).map(int)
p = (lst[0], lst[1])
v = (lst[2], lst[3])
p, v

In [ ]:
res = re.search(r'p=(\d+),(\d+) v=([^,]+),(.+)', xs[0])
lst = L(range(1, 5)).map(res.group).map(int)
p = (lst[0], lst[1])
v = (lst[2], lst[3])
p, v

In [ ]:
def parse(s):
    res = re.search(r'p=(\d+),(\d+) v=([^,]+),(.+)', s)
    lst = L(range(1, 5)).map(res.group).map(int)
    p = (lst[0], lst[1])
    v = (lst[2], lst[3])
    return p, v
parse(xs[0])

In [ ]:
def move(p, v, steps, grid_size):
    x = (p[0] + steps*v[0]) % grid_size[0]
    y = (p[1] + steps*v[1]) % grid_size[1]
    return x, y
move((2,4), (2, -3), 1, (11, 7))

In [ ]:
rs = L(xs).map(parse)
rs100 = rs.starmap(move, steps=100, grid_size=(11, 7))
rs100

In [ ]:
grid_size = (11, 7)
qx_start = [0, grid_size[0] // 2 + 1]
qy_start = [0, grid_size[1] // 2 + 1]
qs = [(x, y) for y in qy_start for x in qx_start]
qs

In [ ]:
def _quadrants(grid_size):
    qx_start = [0, grid_size[0] // 2 + 1]
    qy_start = [0, grid_size[1] // 2 + 1]
    return L((x, y) for y in qy_start for x in qx_start)
_quadrants((11, 7))

In [ ]:
def is_in_quadrant(p, q_start, grid_size):
    in_x = q_start[0] <= p[0] < q_start[0] + grid_size[0] // 2
    in_y = q_start[1] <= p[1] < q_start[1] + grid_size[1] // 2
    return in_x and in_y
is_in_quadrant((0, 2), (0, 0), (11, 7))

In [ ]:
rs100.filter(is_in_quadrant, q_start=qs[0], grid_size=grid_size)

In [ ]:
L(qs).map(lambda q: rs100.filter(is_in_quadrant, q_start=q, grid_size=grid_size))

In [ ]:
L(qs).map(lambda q: len(rs100.filter(is_in_quadrant, q_start=q, grid_size=grid_size)))

In [ ]:
L(qs).map(lambda q: len(rs100.filter(is_in_quadrant, q_start=q, grid_size=grid_size))).product()

In [ ]:
def _safety_factor(ps, grid_size):
    qs = _quadrants(grid_size)
    return qs.map(lambda q: len(ps.filter(is_in_quadrant, q_start=q, grid_size=grid_size))).product()
_safety_factor(rs100, grid_size)

Please combine the functions above into a single function "solve" which will take the input and grid size, it will parse the input, move the robots and compute the safety factor.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's a combined `solve` function that brings everything together:

```python
def solve(inp, grid_size, steps=100):
    # Parse input
    rs = L(inp.splitlines()).map(parse)
    
    # Move all robots
    ps = rs.starmap(move, steps=steps, grid_size=grid_size)
    
    # Calculate safety factor
    return _safety_factor(ps, grid_size)
```

You can test it with:
```python
solve(samp, (11, 7))  # Should return 12
```

Then when it works, try it with your actual input:
```python
solve(inp, (101, 103))
```

Does this look right to you?

In [ ]:
def solve(inp, grid_size, steps=100):
    rs = L(inp.splitlines()).map(parse)
    ps = rs.starmap(move, steps=steps, grid_size=grid_size)
    return _safety_factor(ps, grid_size)

solve(samp, (11, 7))

In [ ]:
solve(samp, (11, 7))  # Should return 12

In [ ]:
solve(inp, (101, 103))

In [ ]:
solve(inp, (101, 103))

# --- Part Two ---

During the bathroom break, someone notices that these robots seem awfully similar to ones built and used at the North Pole. If they're the same type of robots, they should have a hard-coded Easter egg: very rarely, most of the robots should arrange themselves into a picture of a Christmas tree.

What is the fewest number of seconds that must elapse for the robots to display the Easter egg?

I'd like to explore The situation where the robots get to back to the initial state.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Interesting idea! You want to find when the robots return to their starting positions - that's finding the cycle length.

Since each robot moves independently with wrapping, each robot will return to its starting position after some number of steps. The key insight is:

- A robot at position (x, y) with velocity (vx, vy) returns to its start when both x and y coordinates cycle back
- For the x-coordinate, it cycles every `grid_width / gcd(vx, grid_width)` steps
- For the y-coordinate, it cycles every `grid_height / gcd(vy, grid_height)` steps
- Each individual robot cycles at the LCM of these two values
- All robots return to the initial state at the LCM of all individual robot cycles

Would you like to calculate this? You could start by finding the cycle length for a single robot first to see if the logic makes sense.

In [ ]:
from math import gcd,lcm
lcm?

In [ ]:
def _periodicity(v, grid_size):
    per_x = grid_size[0] / gcd(v[0], grid_size[0])
    per_y = grid_size[1] / gcd(v[1], grid_size[1])
    return lcm(int(per_x), int(per_y))

_periodicity((2,0), (10, 10))

In [ ]:
_periodicity((69, 91), (101, 103))

In [ ]:
grid_size = (101, 103)
rs = L(inp.splitlines()).map(parse)
rs
# rs.map(lambda x: x[1])

In [ ]:
grid_size = (101, 103)
rs = L(inp.splitlines()).map(parse)
ps = rs.map(lambda x: x[1]).map(_periodicity, grid_size=grid_size)
lcm(*ps)


In [ ]:
101*103

Write a function which will visualize the input using nonpi array as a grid the function will get positions of each robot and it should plot the positions in the matrix.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's a function to visualize the robot positions on a grid:

```python
def visualize(ps, grid_size):
    grid = np.zeros(grid_size[::-1], dtype=int)
    for p in ps: grid[p[1], p[0]] += 1
    return grid
```

You can test it with:
```python
rs = L(samp.splitlines()).map(parse)
ps = rs.starmap(move, steps=100, grid_size=(11, 7))
visualize(ps, (11, 7))
```

In [ ]:
import numpy as np

def visualize(ps, grid_size):
    grid = np.zeros(grid_size[::-1], dtype=int)
    for p in ps: grid[p[1], p[0]] += 1
    return grid

In [ ]:
rs = L(samp.splitlines()).map(parse)
ps = rs.starmap(move, steps=100, grid_size=(11, 7))
visualize(ps, (11, 7))

Now I want to plot the output of the visualize using matplotlib. The fields with 0 should be white and the other fields should be black.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
import matplotlib.pyplot as plt

def plot_grid(grid):
    plt.figure(figsize=(10, 10))
    plt.imshow(grid > 0, cmap='gray_r', interpolation='nearest')
    plt.axis('off')
    plt.show()
```

Test it with:
```python
plot_grid(visualize(ps, (11, 7)))
```

In [ ]:
import matplotlib.pyplot as plt

def plot_grid(grid, step):
    plt.figure(figsize=(10, 10))
    plt.imshow(grid, cmap='gray_r')
    plt.title(f'Step {step}')
    plt.axis('off')
    plt.show()
plot_grid(visualize(ps, (101, 103)), step=100)

Now I would like to be able to visually inspect multiple grids brighter function that will display multiple grids vertically and horizontally.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
def plot_grids(grids, steps, ncols=5):
    nrows = (len(grids) + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*3, nrows*3))
    axes = axes.flatten() if nrows > 1 or ncols > 1 else [axes]
    for i, (grid, step) in enumerate(zip(grids, steps)):
        axes[i].imshow(grid, cmap='gray_r')
        axes[i].set_title(f'Step {step}')
        axes[i].axis('off')
    for i in range(len(grids), len(axes)): axes[i].axis('off')
    plt.tight_layout()
    plt.show()
```

Test it with:
```python
rs = L(inp.splitlines()).map(parse)
test_steps = [0, 1000, 2000, 3000, 4000, 5000]
grids = [visualize(rs.starmap(move, steps=s, grid_size=(101, 103)), (101, 103)) for s in test_steps]
plot_grids(grids, test_steps, ncols=3)
```

In [ ]:
def plot_grids(grids, steps, ncols=5):
    nrows = (len(grids) + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*3, nrows*3))
    axes = axes.flatten() if nrows > 1 or ncols > 1 else [axes]
    for i, (grid, step) in enumerate(zip(grids, steps)):
        axes[i].imshow(grid, cmap='gray_r')
        axes[i].set_title(f'Step {step}')
        axes[i].axis('off')
    for i in range(len(grids), len(axes)): axes[i].axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
rs = L(inp.splitlines()).map(parse)
test_steps = range(1, 1000)
grids = [visualize(rs.starmap(move, steps=s, grid_size=(101, 103)), (101, 103)) for s in test_steps]
plot_grids(grids, test_steps, ncols=5)

In [ ]:
rs = L(inp.splitlines()).map(parse)
test_steps = range(13, 101*103, 101)
grids = [visualize(rs.starmap(move, steps=s, grid_size=(101, 103)), (101, 103)) for s in test_steps]
plot_grids(grids, test_steps, ncols=5)

In [ ]:
rs = L(inp.splitlines()).map(parse)
test_steps = range(168, 101*103, 103)
grids = [visualize(rs.starmap(move, steps=s, grid_size=(101, 103)), (101, 103)) for s in test_steps]
plot_grids(grids, test_steps, ncols=5)